In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv("text_classification_data.csv")

In [4]:
df.head()

,platform,region,feedback_text,clean_feedback_text,issue_category,star_rating,review_source,urgency,word_count
0,revolut,unknown,getting fed up of the random over reach by sca...,getting fed up of the random over reach by sca...,kyc,2,playstore,Medium,30
1,revolut,unknown,when I urgently needed to send money immediate...,when i urgently needed to send money immediate...,transaction_issues,1,playstore,High,46
2,revolut,unknown,I cannot set to be asked for my permission on ...,i cannot set to be asked for my permission on ...,account_access_&_security_issues,1,playstore,High,25
3,revolut,unknown,8 duplicate transactions from 3 merchants in 1...,8 duplicate transactions from 3 merchants in 1...,cards_&_payment_instruments,1,playstore,High,90
4,revolut,unknown,3 times now someone has tried to fraudulently ...,3 times now someone has tried to fraudulently ...,account_access_&_security_issues,1,playstore,High,53


In [19]:
category = "transaction_issues"

subset = df[df["issue_category"] == category].copy()

subset = subset[subset["clean_feedback_text"].notna()]

subset["clean_feedback_text"] = (
    subset["clean_feedback_text"]
    .str.lower()
    .str.replace(r"[^\w\s]", "", regex=True)
)

words = subset["clean_feedback_text"].str.split().explode()

words = words[words.str.len() > 2]

unigram_counts = (
    words
    .value_counts()
    .reset_index()
    
)

unigram_counts.columns = ["word", "count"]

unigram_counts.head(20)

,word,count
0,the,86
1,and,58
2,money,43
3,you,38
4,this,36
5,app,34
6,they,31
7,for,31
8,but,27
9,with,27


In [21]:
stopwords = {
    "the", "and", "for", "wish", "this", "that", "you", "your", 
    "have", "not", "are", "was", "but", "from", "they"
}

unigram_counts = unigram_counts[
    ~unigram_counts["word"].isin(stopwords)
]

unigram_counts.head(20)

,word,count
2,money,43
5,app,34
9,with,27
10,transfer,27
12,account,25
13,bank,23
16,days,17
17,service,17
19,its,15
20,customer,15
